# Document AI: Fine-tuning LayoutLM for document-parsing using Hugging Face Transformers 


In [ ]:
!sudo apt install -y tesseract-ocr
!pip install pytesseract transformers datasets seqeval

In [1]:
model_id="mrm8488/layoutlm-finetuned-funsd"
processor_id="microsoft/layoutlmv2-base-uncased"
dataset_id ="nielsr/funsd"

In [64]:
from datasets import load_dataset


dataset = load_dataset(dataset_id, split="train[:10%]")
dataset

Reusing dataset funsd (/home/ubuntu/.cache/huggingface/datasets/nielsr___funsd/funsd/1.0.0/8b0472b536a2dcb975d59a4fb9d6fea4e6a1abe260b7fed6f75301e168cbe595)


Dataset({
    features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
    num_rows: 15
})

In [44]:
from transformers import AutoTokenizer, LayoutLMForTokenClassification, LayoutLMv2Processor
import torch

model = LayoutLMForTokenClassification.from_pretrained(model_id)
processor = LayoutLMv2Processor.from_pretrained(processor_id,apply_ocr=False)


In [67]:
from PIL import Image
from functools import partial


def process(sample, processor=None):
    encoding = processor(
        Image.open(sample["image_path"]).convert("RGB"),
        sample["words"],
        boxes=sample["bboxes"],
        word_labels=sample["ner_tags"],
        padding="max_length",
        truncation=True,
    )
    del encoding["image"]
    return encoding


proc_dataset = dataset.map(
    partial(process, processor=processor), remove_columns=["image_path", "words", "ner_tags", "id"]
)
# dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'bbox','lables'])
print(proc_dataset.features.keys())


  0%|          | 0/15 [00:00<?, ?ex/s]

dict_keys(['bboxes', 'input_ids', 'token_type_ids', 'attention_mask', 'bbox', 'labels'])


In [48]:
model(**proc_dataset[0])

IndexError: tuple index out of range